In [1]:
"""
1) Read out the titles from emails_by_address to make two lists of tuples:
(path to file, email the file describes, if its to_ or from_email)
One list will be emails from somebody, the other emails to somebody

2) Use the files from (1) to extract email text from each person 
and create a dict with:
top key: email
sub keys: poi, from_data, to_data
"""

import os
import pickle
import re
from tools.poi_email_addresses import poiEmails

In [2]:
def find_emailpaths(emails):
    """
    Input:
    emails determines what emails to include in output.
    Should be list of tuples with:
    tup[0] = email
    tup[1] = True if email belongs to POI False otherwise
    
    Output:
    List of tuples with:
    tup[0] = path to file with paths organized by email address
    tup[1] = email address
    tup[2] = whether emails were to or from email address
    tup[3] = True if email belongs to POI False otherwise
    """
    
    current_path = os.path.join(os.getcwd(), 'data/emails_by_address')
    all_filenames = os.listdir(current_path)
    emailpath_tuples = []
    
    emails, poi_vals = zip(*emails)

    for filename in all_filenames:
        #generate full path to file
        temp_path = (os.path.join(current_path, filename))
        
        #isolate email from filename and determine if to or from
        if 'from_' in filename:
            temp_filename = filename.replace('from_', '').replace('.txt', '')
            to_from = 'from'
        elif 'to_' in filename:
            temp_filename = filename.replace('to_', '').replace('.txt', '')
            to_from = 'to'
        
        #only include emails from 
        if temp_filename in emails:
            i = emails.index(temp_filename)
            tup = (temp_path, temp_filename, to_from, poi_vals[i])
            emailpath_tuples.append(tup)

    return emailpath_tuples

In [3]:
with open("data/final_project_dataset.pkl", "r") as data_file:
    data_dict = pickle.load(data_file)

In [4]:
#list of emails already in the dataset
data_emails = [(item['email_address'], item['poi']) for item in data_dict.values()]
data_emails = [tup for tup in data_emails if tup[0] != 'NaN']

#additional emails known to belong to pois
file_emails = poiEmails()
file_emails = [(email, True) for email in file_emails]

#all together
all_emails = data_emails + file_emails
print len(all_emails)
all_emails[198]

201


('dan.boyle@enron.com', True)

In [5]:
emailpath_tuples = find_emailpaths(all_emails)

In [6]:
len(set([tup[1] for tup in emailpath_tuples if tup[3]]))

18

In [15]:
from nltk.stem.snowball import SnowballStemmer
import string

def parseOutText(f):
    """ 
    given an opened email file f, return a dict with:
        -sender
        -receivers: [list]
        -original
        -stemmed (original with all words stemmed)
        
    """
    f.seek(0)  ### go back to beginning of file
    full_text = f.read()
    
    # Get metadata
    end_meta = re.compile(r'X-FileName:.*')
    meta_s = end_meta.search(full_text)
    idx = meta_s.end()
    meta = full_text[:idx]
    
    # Get to and from emails from metadata
    def email_extract(raw_string, searched_string):
        result = re.search(raw_string, searched_string)
        emails = result.group(1)
        emails = emails.strip()
        email_list = emails.split(',')
        email_list = [word.strip() for word in email_list if word != '']
        return email_list
    
    from_emails = email_extract(r'From:(.*)', meta)
    to_emails = email_extract(r'To:(.*)', meta)
    cc_emails = email_extract(r'To:(.*)', meta)

    # get content
    content = full_text[idx+1:]
    # remove reply and forward emails
    original = content.split('-----Original Message-----')[0]

    original_alpha = re.sub(r'\W', ' ', original)    # remove non-alpahnumerics
    original_alpha = re.sub(r'\s', ' ', original_alpha) #remove whitespace
    original_letter = re.sub(r'\d', '', original_alpha)
    # stem the words
    original_list = original_letter.split()
    words = ''
    stemmer = SnowballStemmer("english")
    for word in original_list:
        stemmed_word = stemmer.stem(word)
        words = words + stemmed_word + " " 
    dicto = {
            'sender': from_emails,
            'receivers': to_emails,
            'original': original_alpha,
            'stemmed': words
            }
    
    return dicto

In [16]:
email_path = '/Users/TRN/Desktop/1'
with open(email_path, 'r') as email:
    dicto = parseOutText(email)
dicto

{'original': '    Please let me know if you still need Curve Shift   Thanks  Heather  ',
 'receivers': ['k..allen@enron.com'],
 'sender': ['heather.dunton@enron.com'],
 'stemmed': u'pleas let me know if you still need curv shift thank heather '}

In [9]:
def word_dict_maker(emailpath_tuples, temp_save=False):
    """
    Makes word_dict

    Input:
    List of tuples with:
    tup[0] = path to file with paths organized by email address
    tup[1] = email address
    tup[2] = whether emails were to or from email address
    tup[3] = True if email belongs to POI False otherwise

    output:
    word_dict = {
                 'email' : {'poi': True/False
                            'to (or from)': ['sender': from_email,
                                             'receiver': [to_emails],
                                             'original': original_alpha,
                                             'stemmed': words ]
                            }
                 }
    """
    # Assumes maildir directory is in current user Desktop rather than project
    # worspace
    home_path = os.path.expanduser('~')
    maildir_path = os.path.join(home_path, 'Desktop')

    word_dict = {}
    count = 0
    for tup in emailpath_tuples:
        print tup[1], '\n**************\n'

        # Main dict entry for the email address being considered
        if tup[1] not in word_dict:
            word_dict[tup[1]] = {}
            word_dict[tup[1]]['poi'] = tup[3] 
            
        #print word_dict

        # file with paths to emails to/from adress opened
        with open(tup[0], 'r') as email_paths:
            dicto_list = []
            for email_path in email_paths:
                #email_path slice removes a top directory that was removed and newline
                email_path = os.path.join(maildir_path, email_path[20:-1])
                #print email_path
                try:
                    # File with individual email opened and extracted
                    with open(email_path, 'r') as email:
                        #print(email.read())
                        print '.',
                        dicto = parseOutText(email)
                        dicto_list.append(dicto)
                except IOError:
                    print 'ERRROR: ', email_path
                
        if tup[2] == 'from':
            print 'FROM'
            word_dict[tup[1]]['from'] = dicto_list
        elif tup[2] == 'to':
            print 'TO'
            word_dict[tup[1]]['to'] = dicto_list

        #save to file every 100 loops if temp_save=True
        if temp_save:
            count += 1
            if count%100 == 0:
                with open('temp.pkl', 'w') as f:
                    pickle.dump(word_dict, f)
            elif count == len(emailpath_tuples):
                with open('word_dict_subset.pkl', 'w') as f:
                    pickle.dump(word_dict, f)

    return word_dict

In [19]:
test = word_dict_maker(emailpath_tuples)

a..martin@enron.com 
**************

. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . FROM
adam.umanoff@enron.com 
**************

. . . . . . . . . . . . . . . . . . FROM
ben.glisan@enron.com 
**************

. . . . . . . . . . . . . . . . FROM
bill.cordes@enron.com 
**************

. . . . . . . . . . . . FROM
brian.redmond@enron.com 
**************

. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .

. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 

. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 

. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 

. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 

...

 . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .

 . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . TO
tim.despain@enron.com 
**************

. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .

. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 

 . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .

. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 

In [20]:
import pickle

home_path = os.path.expanduser('~')
desktop_path = os.path.join(home_path, 'Desktop')
final = os.path.join(desktop_path, 'new_dict.pkl')

with open(final, 'wb') as f:
    pickle.dump(test, f)

In [ ]:
def email_list_and_labels(word_dict, to_from_all):
    """
    Makes two lists: One with a set of emails and another labeling the set as:
    from poi (1) or non-poi (0)
    """
    emails = []
    labels = []
    choice_dict = {'from': ['from_emails'],
                   'to': ['to_emails'],
                   'all': ['from_emails', 'to_emails']}
    for key in word_dict:
        compilation = []
        for sub_key in choice_dict[to_from_all]:
            for email in word_dict[key][sub_key]:
                compilation.append(email)
        compilation = ''.join(compilation)
        emails.append(compilation)
        label = 0
        if word_dict[key]['poi']:
            label = 1
        labels.append(label)

    return emails, labels

In [ ]:
#test = {key:data_text[key] for key in list(data_text.keys()[:3])}
# all_emails, all_email_labels = email_list_and_labels(data_text, 'all')